In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import requied libraries
from kafka import KafkaConsumer
import pandas as pd
import s3fs
import json
from datetime import datetime

In [2]:
# Kafka consumer config
consumer = KafkaConsumer(
    'crypto_ticker_stream',
    bootstrap_servers=['35.154.244.134:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

In [ ]:
for i in consumer:
    print(i.value)

{'symbol': 'BTCUSDT', 'price': 105480.0, 'price_change': 546.66, 'percent_change': 0.521, 'volume': 7354.34034, 'time': '2025-06-08 12:57:55'}
{'symbol': 'ETHUSDT', 'price': 2515.4, 'price_change': 23.1, 'percent_change': 0.927, 'volume': 224401.3725, 'time': '2025-06-08 12:57:55'}
{'symbol': 'BNBUSDT', 'price': 650.04, 'price_change': 0.84, 'percent_change': 0.129, 'volume': 77403.138, 'time': '2025-06-08 12:57:55'}
{'symbol': 'BTCUSDT', 'price': 105479.99, 'price_change': 546.66, 'percent_change': 0.521, 'volume': 7354.34098, 'time': '2025-06-08 12:57:56'}
{'symbol': 'ETHUSDT', 'price': 2515.4, 'price_change': 23.1, 'percent_change': 0.927, 'volume': 224401.3814, 'time': '2025-06-08 12:57:56'}
{'symbol': 'BTCUSDT', 'price': 105480.0, 'price_change': 546.66, 'percent_change': 0.521, 'volume': 7354.33758, 'time': '2025-06-08 12:57:57'}
{'symbol': 'BTCUSDT', 'price': 105469.46, 'price_change': 525.07, 'percent_change': 0.5, 'volume': 7353.27084, 'time': '2025-06-08 12:58:34'}
{'symbol':

In [ ]:
# Loding the data in S3  
region = 'ap-south-1'
bucket = 'akshay-test1-bucket' 
base_path = 'crypto_data_parquet'
s3 = s3fs.S3FileSystem(client_kwargs={'region_name': region})

batch = []
batch_size = 5
file_count = 0

# Writing file in S3
def write_parquet(batch_data, file_count):
    timestamp_path = datetime.utcnow().strftime('%Y/%m/%d/%H/%M')
    file_path = f's3://{bucket}/{base_path}/{timestamp_path}/batch_{file_count}.parquet'
    df = pd.DataFrame(batch_data)
    df.to_parquet(file_path, index=False, engine='pyarrow', compression='snappy',
                  storage_options={'client_kwargs': {'region_name': region}})
    print(f"Wrote batch {file_count} with {len(df)} records to {file_path}")

for message in consumer:
    batch.append(message.value)
    if len(batch) >= batch_size:
        write_parquet(batch, file_count)
        batch.clear()
        file_count += 1


Wrote batch 0 with 5 records to s3://akshay-test1-bucket/crypto_data_parquet/2025/06/08/07/29/batch_0.parquet
